Importing libraries needed

In [521]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

print("Hello Capstone Project Course! I have libraries")


ModuleNotFoundError: No module named 'geopy'

Getting data from wiki, and adding to the lists

In [522]:
canada_pcodes=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(canada_pcodes, 'html.parser')
PCodeList=[]
BoroughList=[]
NeighbourhoodList=[]

for row in soup.find('table').find_all('tr'):
    cell=row.find_all('td')
    if(len(cell)>0):
        PCodeList.append(cell[0].text.rstrip('\n'))
        BoroughList.append(cell[1].text.rstrip('\n'))
        NeighbourhoodList.append(cell[2].text.rstrip('\n'))
            


Making panda dataframe

In [523]:
dataframe=pd.DataFrame({"PostalCode": PCodeList, "Borough":BoroughList, "Neighborhood":NeighbourhoodList})
dataframe.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Removing "not assigned"

In [524]:
cleanedup_dataframe=dataframe[dataframe.Borough !="Not assigned"].reset_index(drop=True)
cleanedup_dataframe.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Grouping:


In [525]:
grouped_df=cleanedup_dataframe.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x:",".join(x))

grouped_df.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Changing 'not assigned" by Borough items

In [526]:
for index, row in grouped_df.iterrows():
    if row['Neighborhood']=='Not assigned':
        row['Neighborhood']=row['Borough']
        
grouped_df.head()        


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [527]:
grouped_df.shape

(103, 3)

Geting geo data. Yeah, goecoder did not work...

In [528]:
g_df=pd.read_csv('http://cocl.us/Geospatial_data')
g_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging both df together

In [529]:
final=pd.merge(grouped_df, g_df, how='left', left_on='PostalCode', right_on='Postal Code', validate="1:1")
final.drop(labels="Postal Code", axis=1, inplace=True)
final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [530]:
final.shape


(103, 5)

Maps

In [531]:
from geopy.geocoders import Nominatim
# it does not look like this geopy can be imported

ModuleNotFoundError: No module named 'geopy'

In [532]:
CLIENT_ID = 'CO55W55EFAAKYUAZMKJCEL3TDN3R5FJGPVDV0F41MSV0VPQX' # your Foursquare ID
CLIENT_SECRET = '210P3DDGLWPJKFD3AHYJRS0UAW4ILVFBMGT2YX3C30M4U0JH' # your Foursquare Secret
VERSION = '20200615' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CO55W55EFAAKYUAZMKJCEL3TDN3R5FJGPVDV0F41MSV0VPQX
CLIENT_SECRET:210P3DDGLWPJKFD3AHYJRS0UAW4ILVFBMGT2YX3C30M4U0JH


Explore Neighbourhoods

In [533]:
def getNearbyVenues(names, latitudes, longitudes, radius=50):
    
    venues_list=[]
    for  name, lat, lng in zip(names, latitudes, longitudes):
                 
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results =requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print(nearby_venues)
    return(nearby_venues)

In [534]:
LIMIT=100
radius=500
final.head()
venues = getNearbyVenues(names=final['Neighborhood'], latitudes=final['Latitude'],longitudes=final['Longitude'])
venues.head()

                                         Neighborhood  Neighborhood Latitude  \
0                                 Scarborough Village              43.744734   
1                       Steeles West, L'Amoreaux West              43.799525   
2                                          Davisville              43.704324   
3                                          Davisville              43.704324   
4                                          Davisville              43.704324   
5                                          Davisville              43.704324   
6                                            Rosedale              43.679563   
7                                      St. James Town              43.651494   
8                                      St. James Town              43.651494   
9                                      St. James Town              43.651494   
10                           Richmond, Adelaide, King              43.650571   
11                           Richmond, A

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough Village,43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
1,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Price Chopper,43.799445,-79.318563,Grocery Store
2,Davisville,43.704324,-79.388790,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
3,Davisville,43.704324,-79.388790,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café
4,Davisville,43.704324,-79.388790,Positano,43.704558,-79.388639,Italian Restaurant


In [535]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 28 uniques categories.


In [536]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bedford Park, Lawrence Manor East",1,1,1,1,1,1
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",1,1,1,1,1,1
"Commerce Court, Victoria Hotel",5,5,5,5,5,5
Davisville,4,4,4,4,4,4
"First Canadian Place, Underground city",6,6,6,6,6,6
"Kensington Market, Chinatown, Grange Park",4,4,4,4,4,4
"Lawrence Manor, Lawrence Heights",1,1,1,1,1,1
"Little Portugal, Trinity",1,1,1,1,1,1
"Richmond, Adelaide, King",3,3,3,3,3,3


Analyzing

In [537]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Art Gallery,Bakery,Bookstore,Breakfast Spot,Building,Café,Clothing Store,Cocktail Bar,Coffee Shop,...,Japanese Restaurant,Liquor Store,Park,Performing Arts Venue,Playground,Restaurant,Salad Place,Sushi Restaurant,Thrift / Vintage Store,Yoga Studio
0,Scarborough Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,"Steeles West, L'Amoreaux West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Davisville,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [538]:
toronto_onehot.shape

(41, 29)

In [539]:
toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Art Gallery,Bakery,Bookstore,Breakfast Spot,Building,Café,Clothing Store,Cocktail Bar,Coffee Shop,...,Japanese Restaurant,Liquor Store,Park,Performing Arts Venue,Playground,Restaurant,Salad Place,Sushi Restaurant,Thrift / Vintage Store,Yoga Studio
0,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Commerce Court, Victoria Hotel",0.2,0.000000,0.2,0.0,0.000000,0.000000,0.0,0.0,0.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,Davisville,0.0,0.000000,0.0,0.0,0.000000,0.250000,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,"First Canadian Place, Underground city",0.0,0.166667,0.0,0.0,0.166667,0.166667,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0


In [540]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bedford Park, Lawrence Manor East----
                    venue  freq
0        Sushi Restaurant   1.0
1             Art Gallery   0.0
2                  Bakery   0.0
3  Thrift / Vintage Store   0.0
4             Salad Place   0.0


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                    venue  freq
0   Performing Arts Venue   1.0
1             Art Gallery   0.0
2                  Bakery   0.0
3  Thrift / Vintage Store   0.0
4        Sushi Restaurant   0.0


----Commerce Court, Victoria Hotel----
           venue  freq
0    Art Gallery   0.2
1      Bookstore   0.2
2    Coffee Shop   0.2
3  Deli / Bodega   0.2
4            Gym   0.2


----Davisville----
                venue  freq
0                Café  0.25
1         Coffee Shop  0.25
2        Dessert Shop  0.25
3  Italian Restaurant  0.25
4         Art Gallery  0.00


----First Canadian Place, Underground city----
          venue  freq
0      Building  0

In [541]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [542]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building,Café,Clothing Store,Cocktail Bar
1,"CN Tower, King and Spadina, Railway Lands, Har...",Performing Arts Venue,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building,Café,Clothing Store,Cocktail Bar
2,"Commerce Court, Victoria Hotel",Art Gallery,Bookstore,Coffee Shop,Deli / Bodega,Gym,Gastropub,Bakery,Breakfast Spot,Building,Café
3,Davisville,Café,Coffee Shop,Italian Restaurant,Dessert Shop,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building
4,"First Canadian Place, Underground city",Gastropub,Bakery,Salad Place,Building,Café,Cupcake Shop,Yoga Studio,Bookstore,Breakfast Spot,Clothing Store


Kmean clustering for 5 clusters

In [543]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans_toronto = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans_toronto.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 3, 4, 1, 1])

In [544]:
#toronto_merged
toronto_merged=final[final['Neighborhood'].isin(toronto_grouped['Neighborhood'].tolist())].reset_index(drop=True)
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
1,M1W,Scarborough,"Steeles West, L'Amoreaux West",43.799525,-79.318389
2,M4S,Central Toronto,Davisville,43.704324,-79.388790
3,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [545]:
toronto_grouped.head()

,Neighborhood,Art Gallery,Bakery,Bookstore,Breakfast Spot,Building,Café,Clothing Store,Cocktail Bar,Coffee Shop,...,Japanese Restaurant,Liquor Store,Park,Performing Arts Venue,Playground,Restaurant,Salad Place,Sushi Restaurant,Thrift / Vintage Store,Yoga Studio
0,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Commerce Court, Victoria Hotel",0.2,0.000000,0.2,0.0,0.000000,0.000000,0.0,0.0,0.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,Davisville,0.0,0.000000,0.0,0.0,0.000000,0.250000,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,"First Canadian Place, Underground city",0.0,0.166667,0.0,0.0,0.166667,0.166667,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0


Putting together

In [546]:
# add clustering labels
toronto_merged['Cluster Labels']= kmeans_toronto.labels_


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Playground,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building,Café,Clothing Store,Cocktail Bar
1,M1W,Scarborough,"Steeles West, L'Amoreaux West",43.799525,-79.318389,1,Grocery Store,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building,Café,Clothing Store,Cocktail Bar
2,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Café,Coffee Shop,Italian Restaurant,Dessert Shop,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building
3,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building,Café,Clothing Store,Cocktail Bar
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Breakfast Spot,Japanese Restaurant,Coffee Shop,Yoga Studio,Gastropub,Bakery,Bookstore,Building,Café,Clothing Store
